In [23]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, load_metric
from torch.utils.data import DataLoader
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import numpy as np

os.getcwd()
#os.chdir('c:\\Users\\Kirchner\\Desktop\\Masterarbeit')

'/local/home/CE/svenk'

In [36]:

import gc

gc.collect()

torch.cuda.empty_cache()

In [24]:
# train_size = 50
# def read_annotations(txt_name, mode):
#     texts = []
#     labels = []

#     with open (txt_name, encoding = "utf-8", mode = "r+") as f:
#         for line in f.readlines():
#             texts.append(line.split("\t")[8])
#             if mode == "opinion":
#                 labels.append(line.split("\t")[5])
#             elif mode == "sentiment":
#                 labels.append(line.split("\t")[6])
#             else:
#                 print(f"There is no label for {mode}")

#     return texts,labels

In [3]:
# meine_texte, meine_labels  = read_annotations("cleaned_annotated_data_training.txt", "sentiment")

# texts_train = meine_texte[:train_size]
# texts_test = meine_texte[train_size:]
# labels_train = meine_labels[:train_size]
# labels_test = meine_labels[train_size:]

# print(len(labels_train), len(labels_test))

In [37]:
default = True
if os.path.isdir("meine_tokenizer") and not default:
    print("Loading finetuned tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained("meine_tokenizer_10E_sentiment_500")
else:
    tokenizer = AutoTokenizer.from_pretrained("oliverguhr/german-sentiment-bert")

if os.path.isdir("Sentiment_Bert_mit_spiegel") and not default:
    print("Loading finetuned model...")
    model = AutoModelForSequenceClassification.from_pretrained("Sentiment_Bert_mit_spiegel_10E_sentiment_500")
else:
    model = AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert")

In [39]:
text_train = []
labels_train = []
text_test = []
labels_test = []

label_to_id = {"positive" : 0, "negative" : 1, "neutral" : 2}
id_to_label = {0:"positive", 1:"negative", 2:"neutral"}

with open ("cleaned_annotated_data_training_800.txt", encoding="utf-8", mode="r+") as f:
    training_data = f.readlines()[:800]
    split_index = int(len(training_data)*0.8)

    for line in training_data[:88]:
        text_train.append(line.split("\t")[8])
        labels_train.append(label_to_id[line.split("\t")[6]])
    for line in training_data[89:split_index]:
        text_train.append(line.split("\t")[9])
            
        labels_train.append(label_to_id[line.split("\t")[6]])
    for line in training_data[split_index+1:]:
        text_test.append(line.split("\t")[9])
        labels_test.append(label_to_id[line.split("\t")[6]])

train_dict = {"texts": text_train, "labels" : labels_train}
split_index

624

In [40]:
for idx, text in enumerate(train_dict["texts"]):
    train_dict["texts"][idx] = text.rstrip("\n")

len(train_dict["texts"])
    



623

In [41]:
train_text_tokenized = tokenizer(text_train, truncation=True, padding=True)

test_text_tokenized = tokenizer(text_test, truncation=True, padding=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [42]:
def preprocess_function(examples):
    return tokenizer(examples["texts"], padding = True, truncation=True)


In [25]:
# spiegel = SpiegelDataset("cleaned_annotated_data_training.txt")

# spiegel_data = load_dataset("text", data_files={"train": "Spiegel_train.txt", "test" : "Spiegel_test.txt"})

Using custom data configuration default-b87b2cbca73061b8
Reusing dataset text (C:\Users\Kirchner\.cache\huggingface\datasets\text\default-b87b2cbca73061b8\0.0.0\e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
100%|██████████| 2/2 [00:00<00:00, 795.81it/s]


In [43]:

test_dict = {"texts" : text_test,
            "labels" : labels_test}
            

metric = load_metric("accuracy")

def compute_metrics_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print("Predictions", predictions, "Labels", labels)
    return metric.compute(predictions=predictions, references=labels)


meine_dataset_train = Dataset.from_dict(train_dict)

mein_dataset_test = Dataset.from_dict(test_dict)


tokenized_dataset_train = meine_dataset_train.map(preprocess_function, batched= True)
tokenized_dataset_test = mein_dataset_test.map(preprocess_function, batched=True)

In [44]:
len(tokenized_dataset_train)


623

In [46]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=20,
    evaluation_strategy="epoch"
    
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics = compute_metrics_accuracy
)

# trainer.train()
trainer.evaluate()


W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.


Predictions [1 1 1 1 2 2 2 1 1 2 2 2 2 1 1 2 1 2 2 2 1 2 1 1 1 0 2 2 2 1 2 1 1 2 1 2 1
 2 2 1 2 1 1 2 1 2 2 1 0 1 2 2 2 1 1 1 1 2 1 1 1 2 1 2 1 2 1 1 1 1 1 2 1 0
 2 1 1 1 2 2 2 1 1 2 1 2 2 1 1 2 1 2 2 2 2 1 1 2 2 1 1 2 1 1 2 1 2 2 1 2 2
 2 1 1 1 1 1 1 1 1 1 1 2 2 2 2 1 2 1 1 1 2 1 2 2 2 1 1 1 1 2 1 2 1 0 1 1 2
 2 1 2 2 2 2 2 2] Labels [1 1 2 1 2 1 2 2 1 1 2 2 2 1 1 2 1 1 2 1 1 1 1 1 1 2 2 1 2 1 2 1 1 1 1 1 1
 1 2 1 2 2 1 2 1 2 1 1 1 2 1 2 2 1 1 2 1 2 2 1 1 2 1 0 1 2 1 2 1 1 1 2 1 0
 0 1 1 1 0 2 1 1 2 2 2 1 1 1 1 1 2 2 1 1 2 2 1 2 1 1 1 1 1 1 1 1 2 1 1 2 2
 1 1 1 2 2 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 2 1 1 1 2 1 1 1 2 2 2 1 2 2 2 1 1
 2 1 1 1 1 0 0 1]


{'eval_loss': 2.9816603660583496,
 'eval_accuracy': 0.6153846153846154,
 'eval_runtime': 5.0742,
 'eval_samples_per_second': 30.744}

In [12]:
torch.cuda.get_device_name(0)

'GeForce GTX TITAN X'

In [47]:

model.save_pretrained("Sentiment_Bert_mit_spiegel_10E_sentiment_800_2e-5")
tokenizer.save_pretrained("meine_tokenizer_10E_sentiment_800_2e-5")


('meine_tokenizer_10E_sentiment_800_2e-5/tokenizer_config.json',
 'meine_tokenizer_10E_sentiment_800_2e-5/special_tokens_map.json',
 'meine_tokenizer_10E_sentiment_800_2e-5/vocab.txt',
 'meine_tokenizer_10E_sentiment_800_2e-5/added_tokens.json')

In [32]:
test_dict["labels"]

[1,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2]

In [68]:
len(train_text_tokenized["attention_mask"])
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [143]:


# training_args = TrainingArguments(output_dir="./results",
#                     learning_rate=2e-5,
#                     per_device_eval_batch_size=4,
#                     per_device_train_batch_size=4,
#                     num_train_epochs=3,
#                     weight_decay=0.01
#                     )

# trainer = Trainer(
#             model=model,
#             args=training_args,
#             train_dataset=train_text_tokenized,
#             eval_dataset=test_text_tokenized,
#             tokenizer = tokenizer,
#             data_collator= data_collator
# )


# trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 3
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3





AttributeError: 'list' object has no attribute 'keys'

In [5]:
from germansentiment import SentimentModel

['Nachtrag: Dieser Artikel ist wirklich enttäuschend. Dass es manchem Leser vielleicht schwerfällt, sich an den Ablauf der Ereignisse zu erinnern, oder diese in einen Kontext zu setzen, okay. Aber der Spiegel? Da könnte man ja wirklich vermuten, das sei Click-Baiting, um Seitenaufrufe zu generieren, oder vom US-Außenministerium abgeschrieben. Wahrscheinlich aber doch einfach nur Naivität und leichte Beeinflussbarkeit der Autoren.\n',
 'Ein Flugzeug aus China macht noch keinen Frühling. Die Kritik dieses Artikels ist völlig unangebracht. Die EU hat gesundheitspolitisch fast keine Befugnisse. Das ist so gewollt und an sich auch sehr vorteilhaft. Koordinierte politische Aktionen auf europäischer Ebene würden hier gar nicht weiterhelfen. Jede Region in Europa trifft die für sie und der Lage angemessenen Entscheidungen. Das gibt dann kein einheitliches Bild ab, aber der Virus tritt auch nicht überall einheitlich auf. Die einzige Ebene auf der ein Austausch wirklich erforderlich ist, ist die

In [24]:
model = SentimentModel()
all = 0
correct = 0

hello = [id_to_label[x] for x in train_dict["labels"][:150]]

for idx, word in enumerate(model.predict_sentiment(train_dict["text"][:150])):
    print(word, idx)
    all += 1
    if hello[idx] == word:
        correct+= 1

print(correct/all)


RuntimeError: CUDA out of memory. Tried to allocate 800.00 MiB (GPU 0; 4.00 GiB total capacity; 1.90 GiB already allocated; 317.91 MiB free; 2.53 GiB reserved in total by PyTorch)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_imdb = imdb.map(preprocess_function, batched=True)